In [639]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [640]:
file = open('abalone.data')

lines = file.readlines()

dataset = []

for line in lines:
    var = line.split(',')
    var = [x.split('\n')[0] for x in var]
    dataset.append(var)

In [641]:
dataFrame = pd.DataFrame(dataset,columns=["Sex","Length","Diameter","Height","Whole weight","Shucked weight","Viscera weight","Shell weight","Rings"])

for i in range(1,len(dataFrame.columns)):
    dataFrame[dataFrame.columns[i]] = pd.to_numeric(dataFrame[dataFrame.columns[i]])
    
unique,arr = np.unique(dataFrame.Rings,return_counts=True)
index = np.argmax(arr)
print(unique[index])

9


In [642]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(np.array(dataFrame.Sex).reshape(-1,1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [643]:
transformed_sex = one_hot_encoder.transform(np.array(dataFrame.Sex).reshape(-1,1)).toarray()

In [644]:
new_df = pd.DataFrame(transformed_sex,columns=one_hot_encoder.get_feature_names())
new_df

,x0_F,x0_I,x0_M
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,0.0,1.0,0.0
...,...,...,...
4172,1.0,0.0,0.0
4173,0.0,0.0,1.0
4174,0.0,0.0,1.0
4175,1.0,0.0,0.0


In [645]:
dataFrame = dataFrame.drop(columns=['Sex'])
dataFrame = pd.concat((new_df,dataFrame),axis=1)

# dataFrame = dataFrame.groupby('Rings').apply(lambda x: x.sample(1))

In [646]:
dataFrame = dataFrame.sample(frac=1).reset_index(drop=True)

X = dataFrame.drop(columns="Rings")
y = dataFrame.Rings

In [647]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2)

## Model : KNeighboursClassifier 

In [648]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier()
knc.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [649]:
knc.score(X_train,y_train)

0.45974259203831186

In [650]:
knc.score(X_test,y_test)

0.2236842105263158

In [651]:
y_pred = knc.predict(X_test)

results = np.vstack((y_pred,y_test)).transpose()

results = pd.DataFrame(results,columns=["Prediction","Actual"])
results

,Prediction,Actual
0,9,10
1,12,9
2,10,11
3,11,10
4,9,12
...,...,...
831,10,22
832,8,7
833,8,6
834,9,12


## Model : RandomForest 

In [652]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [653]:
rfc.score(X_train,y_train)

1.0

In [654]:
rfc.score(X_test,y_test)

0.2452153110047847

In [655]:
y_pred = rfc.predict(X_test)

results = np.vstack((y_pred,y_test)).transpose()

results = pd.DataFrame(results,columns=["Prediction","Actual"])
results

,Prediction,Actual
0,11,10
1,11,9
2,11,11
3,9,10
4,11,12
...,...,...
831,15,22
832,8,7
833,6,6
834,9,12


In [656]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [657]:
dtc.score(X_train,y_train)

1.0

In [658]:
dtc.score(X_test,y_test)

0.20454545454545456

In [659]:
y_pred = dtc.predict(X_test)

results = np.vstack((y_pred,y_test)).transpose()

results = pd.DataFrame(results,columns=["Prediction","Actual"])
results

,Prediction,Actual
0,9,10
1,7,9
2,11,11
3,9,10
4,14,12
...,...,...
831,11,22
832,8,7
833,8,6
834,13,12
